In [14]:
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from numpy import asarray
from numpy import mean
from numpy import std
from xgboost import XGBRegressor
import xgboost
from pathlib import Path
import time
from sklearn.gaussian_process import GaussianProcessRegressor

0.7821003944033458 => sequenceLength:10, nTrees: 64, max_depth: 50

0.7944474821071543 => sequenceLength:10, nTrees: 64, max_depth: None

0.7905605781030345 => sequenceLength:10, nTrees: 64, max_depth: None, labels MinMax scaled

0.7893843537936445 => sequenceLength:10, nTrees: 64, max_depth: None, labels standard scaled

0.8101881801103619 => sequenceLength:20, nTrees: 64, max_depth: None, labels standard scaled

### XGBoost (double model)
* 0.7740843741300616, 0.6670367841427146 => sequenceLength:20, nTrees: 64, max_depth: default, labels standard scaled
* 0.7339056408411957, 0.6153000034595998 => sequenceLength:20, nTrees: 64, max_depth: default, labels standard scaled, gpu_hist
* 0.8160022012880737, 0.7179601372210322 => sequenceLength:20, nTrees: 64, max_depth: 10, labels standard scaled, gpu_hist
* 0.84167863251325, 0.7641087580768761 => sequenceLength:20, nTrees: 64, max_depth: 20, labels standard scaled, gpu_hist

### RandomForest (double model)
* 0.83827540937785, 0.7663303903141425 => sequenceLength:20, nTrees: 32, max_depth: 50, labels standard scaled
* 0.8433744923454447,0.7731579198827838 => sequenceLength:20, nTrees: 64, max_depth: 50, labels standard scaled
* 0.8430462022005601, 0.7724669773855409 => sequenceLength:20, nTrees: 64, max_depth: 50, labels standard scaled, bootstrap=True
#### After Introducing trajectory() features
##### RF
* 0.9832796800593451, 0.9871431958518339 => sequenceLength:20, nTrees: 64, max_depth: 50, labels standard scaled, bootstrap=True
##### XG
* 0.9782034741965696, 0.9856036857164658 => sequenceLength:20, nTrees: 64, max_depth: 20, labels standard scaled, bootstrap=True
* 0.9801315191585513, 0.9866471744961971 => sequenceLength:20, nTrees: 64, max_depth: 20, labels standard scaled, bootstrap=True, learning_rate=0.2
* 0.9818559205025458, 0.9871650521809248 => sequenceLength:20, nTrees: 64, max_depth: 20, labels standard scaled, bootstrap=True, learning_rate=0.1

## Load Data

In [2]:
use_windows = True
if use_windows:
    dbfile = open('otherLargeFiles/CNN-dataset.pkl', 'rb')     
else:
    dbfile = open('otherLargeFiles/CNN-dataset-no-window.pkl', 'rb')     
dataset = pickle.load(dbfile)
dbfile.close()

# dataset["samples"] = dataset["samples"][:,:,(0,1,2,5,7,9,10,11,12,13,14,15,16)]
dataset["samples"] = dataset["samples"][:,:,(0,4,5,6,7,8,9,10,11,12)]
if len(dataset["samples"].shape) == 3:
    dataset["samples"] = dataset["samples"].reshape(dataset["samples"].shape[0],dataset["samples"].shape[1]*dataset["samples"].shape[2])
    
    
testDriveFile = open('otherLargeFiles/test-drive-0.pkl', 'rb')
testDrive = pickle.load(testDriveFile)
testDriveFile.close()
# testDrive["samples"] = testDrive["samples"][:,:,(0,1,2,5,7,9,10,11,12,13,14,15,16)]
testDrive["samples"] = testDrive["samples"][:,:,(0,4,5,6,7,8,9,10,11,12)]
if len(testDrive["samples"].shape) == 3:
    testDrive["samples"] = testDrive["samples"].reshape(testDrive["samples"].shape[0],testDrive["samples"].shape[1]*testDrive["samples"].shape[2])

## Split into training/validation

In [3]:
trainInputs, testInputs, trainLabels, testLabels = train_test_split(dataset["samples"], dataset["labels"], test_size=0.5, shuffle=True)

# The Model

In [66]:
trainInputs.shape

(272419, 130)

In [118]:
rfLongModel = RandomForestRegressor(n_estimators=64, max_depth=50, verbose=100, n_jobs=-1, bootstrap=True)
rfLatModel = RandomForestRegressor(n_estimators=64, max_depth=50, verbose=100, n_jobs=-1, bootstrap=True)

rfLongModel.fit(trainInputs, trainLabels[:,0])
rfLatModel.fit(trainInputs, trainLabels[:,1])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
building tree 1 of 64building tree 2 of 64
building tree 3 of 64
building tree 4 of 64
building tree 5 of 64
building tree 6 of 64

building tree 7 of 64
building tree 8 of 64
building tree 9 of 64[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min

building tree 10 of 64[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.1min

building tree 11 of 64[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.1min

building tree 12 of 64[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.1min

building tree 13 of 64[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.2min

building tree 14 of 64[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.2min

building tree 15 of 64[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.2min

building tree 16 of 64[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  1.2min

building tree 17 of 64[Parallel(n_jobs=-1)]: Done   9 tasks     

RandomForestRegressor(max_depth=50, n_estimators=64, n_jobs=-1, verbose=100)

In [6]:
startTime = time.time()
bpLongModel = BartpyRegressor()
bpLatModel = BartpyRegressor()
bpLongModel.fit(trainInputs[:], trainLabels[:,0])
bpLatModel.fit(trainInputs[:], trainLabels[:,1])
endTime = time.time()
timeElapsed = endTime - startTime
print(timeElapsed)

106.4653697013855


In [7]:
print(gaussianLongModel.score(testInputs, testLabels[:,0]))
print(gaussianLatModel.score(testInputs, testLabels[:,1]))

-992.9873073815288
-1380.831634263416


In [50]:
startTime = time.time()
xgbLongModel = XGBRegressor(eta=0.1, n_estimators=32, max_depth=20, verbosity=3, n_jobs=-1, objective='reg:squarederror', tree_method="gpu_hist")
xgbLatModel = XGBRegressor(eta=0.1, n_estimators=32, max_depth=20, verbosity=3, n_jobs=-1, objective='reg:squarederror', tree_method="gpu_hist")
xgbLongModel.fit(trainInputs, trainLabels[:,0])
xgbLatModel.fit(trainInputs, trainLabels[:,1])
endTime = time.time()
timeElapsed = endTime - startTime
print(timeElapsed)

[18:12:15] DEBUG: ../src/tree/updater_gpu_hist.cu:899: [GPU Hist]: Configure
[18:12:15] DEBUG: ../src/common/device_helpers.cu:38: Running nccl init on: 10.0
[18:12:15] ======== Monitor: SketchContainer ========
[18:12:15] MakeCuts: 0.000279s, 1 calls @ 279us

[18:12:15] Prune: 0.000202s, 1 calls @ 202us

[18:12:15] ScanInput: 0.00256s, 1 calls @ 2560us

[18:12:15] Unique: 0.00016s, 1 calls @ 160us

[18:13:35] ======== Monitor: Learner ========
[18:13:35] Configure: 0.000575s, 1 calls @ 575us

[18:13:35] EvalOneIter: 0.00034s, 32 calls @ 340us

[18:13:35] GetGradient: 0.002247s, 32 calls @ 2247us

[18:13:35] PredictRaw: 0.000257s, 32 calls @ 257us

[18:13:35] UpdateOneIter: 77.7539s, 32 calls @ 77753942us

[18:13:35] ======== Monitor: GBTree ========
[18:13:35] BoostNewTrees: 77.7503s, 32 calls @ 77750319us

[18:13:35] CommitModel: 2.8e-05s, 32 calls @ 28us

[18:13:35] ======== Device 0 Memory Allocations:  ========
[18:13:35] Peak memory usage: 1957MiB
[18:13:35] Number of allocations

# Results

In [51]:
print(xgbLongModel.score(testInputs, testLabels[:,0]))
print(xgbLatModel.score(testInputs, testLabels[:,1]))

print(xgbLongModel.score(trainInputs, trainLabels[:,0]))
print(xgbLatModel.score(trainInputs, trainLabels[:,1]))

[18:21:32] DEBUG: ../src/tree/updater_gpu_hist.cu:899: [GPU Hist]: Configure
[18:21:32] ======== Device 0 Memory Allocations:  ========
[18:21:32] Peak memory usage: 1957MiB
[18:21:32] Number of allocations: 11915518
[18:21:32] ======== Monitor:  ========
[18:21:32] ======== NCCL Statistics========
[18:21:32] AllReduce calls: 0
[18:21:32] AllReduce total MiB communicated: 0
0.8998329988351264
[18:21:32] DEBUG: ../src/tree/updater_gpu_hist.cu:899: [GPU Hist]: Configure
[18:21:32] ======== Device 0 Memory Allocations:  ========
[18:21:32] Peak memory usage: 1957MiB
[18:21:32] Number of allocations: 11915518
[18:21:32] ======== Monitor:  ========
[18:21:32] ======== NCCL Statistics========
[18:21:32] AllReduce calls: 0
[18:21:32] AllReduce total MiB communicated: 0
0.9124641369929349
0.986538910330104
0.9923920950347264


In [141]:
print(rfLongModel.score(testInputs, testLabels[:,0]))
print(rfLatModel.score(testInputs, testLabels[:,1]))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:    0.5s
[Para

In [76]:
print(rfLongModel.score(trainInputs, trainLabels[:,0]))
print(rfLatModel.score(trainInputs, trainLabels[:,1]))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:    0.8s
[Para

In [12]:
testInputs.shape

(228404, 130)

In [36]:
testDrivePaths = [str(path) for path in Path("./otherLargeFiles").rglob("test-drive*.pkl")]
testDrives = []
for testDrivePath in testDrivePaths:
    testDriveFile = open(testDrivePath, 'rb')
    testDrive = pickle.load(testDriveFile)
    testDriveFile.close()
    testDrive["samples"] = testDrive["samples"][:,:,(0,1,2,5,7,9,10,11,12,13,14,15,16)]
    if len(testDrive["samples"].shape) == 3:
        testDrive["samples"] = testDrive["samples"].reshape(testDrive["samples"].shape[0],testDrive["samples"].shape[1]*testDrive["samples"].shape[2])
    testDrives.append(testDrive)

In [41]:
len(testDrives)

51

In [43]:
testDriveScores = []
for testDrive in testDrives:
    longScore = xgbLongModel.score(testDrive["samples"], testDrive["labels"][:,0])
    latScore = xgbLatModel.score(testDrive["samples"], testDrive["labels"][:,1])
    testDriveScores.append([longScore, latScore])
testDriveScores = np.array(testDriveScores)

In [44]:
pd.DataFrame({"long":testDriveScores[:,0], "lat":testDriveScores[:,1]}).describe()

,long,lat
count,51.000000,51.000000
mean,-0.739904,-0.083938
std,1.920046,1.212332
min,-8.243637,-4.453251
25%,-2.006684,-0.342005
50%,-0.179089,0.204119
75%,0.821230,0.899429
max,0.981760,0.977233


## Leave One Out

In [ ]:
drives = {}
for drivePath in Path("./otherLargeFiles/AllDrives").rglob("drive*.pkl"):
    driveFile = open(drivePath, 'rb')
    drive = pickle.load(driveFile)
    driveFile.close()
    driveID = "-".join(str(drivePath).split("/")[-1].split(".")[0].split("-")[1:])
    drives[driveID] = drive

results = {"leftoutID":[], "scores":[]}
startTime = time.time()
for i,leftoutID in enumerate(tuple(drives.keys())):
    sampleSets = []
    labelSets = []
    for driveID in drives.keys():
        if driveID != leftoutID:
            sampleSets.append(drives[driveID]["samples"])
            labelSets.append(drives[driveID]["labels"])
    trainInputs = np.concatenate(sampleSets)
    trainLabels = np.concatenate(labelSets)
    testInputs = drives[leftoutID]["samples"]
    testLabels = drives[leftoutID]["labels"]
    trainInputs = trainInputs.reshape(trainInputs.shape[0],trainInputs.shape[1]*trainInputs.shape[2])
    testInputs = testInputs.reshape(testInputs.shape[0],testInputs.shape[1]*testInputs.shape[2])
    print("Traing round", i, "of", len(tuple(drives.keys())))
    xgbLongModel = XGBRegressor(eta=0.1, n_estimators=32, max_depth=20, verbosity=1, n_jobs=-1, objective='reg:squarederror', tree_method="gpu_hist")
    xgbLatModel = XGBRegressor(eta=0.1, n_estimators=32, max_depth=20, verbosity=1, n_jobs=-1, objective='reg:squarederror', tree_method="gpu_hist")
    xgbLongModel.fit(trainInputs, trainLabels[:,0])
    xgbLatModel.fit(trainInputs, trainLabels[:,1])
    results["leftoutID"].append(leftoutID)
    scores = [xgbLongModel.score(testInputs, testLabels[:,0]), xgbLatModel.score(testInputs, testLabels[:,1])]
    results["scores"].append(scores)
    print("Finished training round", i, "of", len(tuple(drives.keys())))
    print("Scores on left out drive:",scores[0],scores[1])
    print("Time Elapsed:",time.time()-startTime)

In [72]:
resultDataframe = pd.DataFrame({"LeftoutID": results["leftoutID"], "LongScore":np.array(results["scores"])[:,0], "LatScore":np.array(results["scores"])[:,1]})

In [76]:
resultFile = open("otherLargeFiles/LOOResults.pkl","ab")
pickle.dump(resultDataframe, resultFile)
resultFile.close()

In [3]:
rf = open("otherLargeFiles/LOOResults.pkl","rb")
r = pickle.load(rf)
rf.close()

In [4]:
r.describe()

,LongScore,LatScore
count,179.000000,179.000000
mean,-2.012072,-1.510191
std,14.902567,12.265310
min,-199.633592,-161.338284
25%,-1.412281,-0.833929
50%,-0.633363,-0.153860
75%,-0.012384,0.434011
max,0.815398,0.913618
